In [147]:
import pandas as pd
from sklearn.cluster import KMeans
import requests 
import numpy as np
from joblib import load


In [148]:
# solicitacoes_credito = pd.read_excel('saidas/solicitacoes_credito_encoded.xlsx')
solicitacoes_credito = pd.read_excel('saidas/solicitacoes_credito_clusters.xlsx')


# Carregar o modelo KMeans salvo
kmeans_model = load('saidas/Modelo_Kmeans.joblib')

In [149]:
# # Carregar limites de crédito
# solicitacoes_credito = pd.read_pickle('saidas/solicitacoes_credito_clusters.pkl')
# solicitacoes_credito['credit_limit'] = 1000 * (solicitacoes_credito['score_atraso'] / solicitacoes_credito['score_atraso'].max())
# credit_limits_by_cluster = solicitacoes_credito.groupby('cluster')['credit_limit'].mean().reset_index()
# credit_limits_by_cluster.columns = ['cluster', 'recommended_credit_limit']
# credit_limits_by_cluster.set_index('cluster', inplace=True)

In [150]:
# def preprocess_customer_data(data):
#     # Aqui você deve incluir o pré-processamento dos dados, como normalização e tratamento de dados ausentes
#     return np.array([data])


In [151]:
# def recommend_credit_limit(customer_data):
#     # Pré-processar os dados do cliente
#     preprocessed_data = preprocess_customer_data(customer_data)
    
#     # Prever o cluster do cliente usando o modelo carregado
#     cluster = kmeans_model.predict(preprocessed_data)[0]
    
#     # Obter o limite de crédito para o cluster previsto
#     recommended_limit = credit_limits_by_cluster[credit_limits_by_cluster['cluster'] == cluster]['recommended_credit_limit'].values[0]
    
#     return recommended_limit


Entrada de novos clientes 

In [152]:
# Carregar a base de novos clientes do arquivo Excel
novos_clientes = pd.read_excel('saidas/novos_clientes.xlsx')

In [153]:
# Certificar-se de que a base de novos clientes tem as colunas necessárias
expected_columns = [
    'maiorAtraso', 'prazoMedioRecebimentoVendas', 'titulosEmAberto',
    'valorSolicitado', 'status', 'valorAprovado', 'periodoBalanco',
    'ativoCirculante', 'passivoCirculante', 'totalAtivo',
    'totalPatrimonioLiquido', 'endividamento', 'duplicatasAReceber',
    'estoque', 'faturamentoBruto', 'margemBruta', 'custos',
    'intervaloFundacao', 'capitalSocial', 'limiteEmpresaAnaliseCredito',
    'score_atraso'
]

In [154]:
# Garantir que todas as colunas esperadas estão presentes
if not set(expected_columns).issubset(novos_clientes.columns):
    raise ValueError("A base de novos clientes não contém todas as colunas esperadas.")


In [155]:
# Preencher possíveis valores ausentes se necessário (exemplo)
novos_clientes.fillna(0, inplace=True)  # Ajuste conforme necessário

# Converter colunas categóricas para numéricas
# Por exemplo, convertendo a coluna 'status' para numérico
if 'status' in novos_clientes.columns:
    novos_clientes['status'] = novos_clientes['status'].apply(lambda x: 1 if x == 'Aprovado' else 0)

# Selecionar apenas as colunas esperadas
X_novos_clientes = novos_clientes[expected_columns].values


In [156]:
# Prever o cluster para cada novo cliente
try:
    clusters_novos_clientes = kmeans_model.predict(X_novos_clientes)
except Exception as e:
    print(f"Erro ao prever clusters: {e}")
    clusters_novos_clientes = np.full(X_novos_clientes.shape[0], np.nan)  # Usar NaN para falhas


C:\Users\laris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [157]:
# Calcular limites de crédito recomendados
# Carregar limites de crédito recomendados por cluster
solicitacoes_credito = pd.read_pickle('saidas/solicitacoes_credito_clusters.pkl')
solicitacoes_credito['credit_limit'] = 1000 * (solicitacoes_credito['score_atraso'] / solicitacoes_credito['score_atraso'].max())
credit_limits_by_cluster = solicitacoes_credito.groupby('cluster')['credit_limit'].mean().reset_index()
credit_limits_by_cluster.columns = ['cluster', 'recommended_credit_limit']


In [158]:
# Mapear limites de crédito para clusters
credit_limit_map = credit_limits_by_cluster.set_index('cluster')['recommended_credit_limit'].to_dict()

In [159]:
# Adicionar coluna de limite de crédito aos novos clientes
novos_clientes['cluster'] = clusters_novos_clientes
novos_clientes['credito_recomendado'] = novos_clientes['cluster'].map(credit_limit_map)

In [160]:
novos_clientes.to_excel('saidas/recomendacao.xlsx')

In [161]:
# # Função para recomendar limite de crédito com base no cluster
# def recommend_credit_limit(customer_data, model, credit_limits_by_cluster):
#     # Prever o cluster do cliente
#     cluster = model.predict([customer_data])[0]
    
#     # Encontrar o limite de crédito recomendado para o cluster
#     recommended_limit = credit_limits_by_cluster[credit_limits_by_cluster['cluster'] == cluster]['recommended_credit_limit'].values[0]

In [162]:
# # Exemplo de cliente
# # Observação: customer_data deve ter o mesmo formato e escala das features usadas no treinamento do modelo
# customer_data = [0.5, 0.4, 0.3, 0.6, 0.7, 0.5, 0.4]  # Exemplo de dados normalizados do cliente
# recommended_limit = recommend_credit_limit(customer_data, km, credit_limits_by_cluster)

# print(f"Recommended Credit Limit: {recommended_limit}")

# print("Limites de crédito recomendados por cluster:")
# print(credit_limits_by_cluster)

In [163]:
# import pandas as pd
# import numpy as np

# # Definir a quantidade de novos clientes fictícios
# num_novos_clientes = 10

# # Criar uma base de novos clientes com os campos especificados
# novos_clientes = pd.DataFrame({
#     'maiorAtraso': np.random.randint(0, 60, num_novos_clientes),
#     'prazoMedioRecebimentoVendas': np.random.randint(0, 120, num_novos_clientes),
#     'titulosEmAberto': np.random.uniform(0, 500000, num_novos_clientes),
#     'valorSolicitado': np.random.uniform(50000, 2000000, num_novos_clientes),
#     'status': np.random.choice(['Aprovado', 'Reprovado', 'Em análise'], num_novos_clientes),
#     'valorAprovado': np.random.uniform(50000, 1500000, num_novos_clientes),
#     'periodoBalanco': np.random.randint(1, 12, num_novos_clientes),
#     'ativoCirculante': np.random.uniform(1000000, 30000000, num_novos_clientes),
#     'passivoCirculante': np.random.uniform(500000, 20000000, num_novos_clientes),
#     'totalAtivo': np.random.uniform(1000000, 50000000, num_novos_clientes),
#     'totalPatrimonioLiquido': np.random.uniform(500000, 40000000, num_novos_clientes),
#     'endividamento': np.random.uniform(0, 1, num_novos_clientes),
#     'duplicatasAReceber': np.random.uniform(0, 2000000, num_novos_clientes),
#     'estoque': np.random.uniform(0, 1000000, num_novos_clientes),
#     'faturamentoBruto': np.random.uniform(1000000, 100000000, num_novos_clientes),
#     'margemBruta': np.random.uniform(-5000000, 50000000, num_novos_clientes),
#     'custos': np.random.uniform(0, 50000000, num_novos_clientes),
#     'intervaloFundacao': np.random.randint(1, 100, num_novos_clientes),
#     'capitalSocial': np.random.uniform(10000, 5000000, num_novos_clientes),
#     'limiteEmpresaAnaliseCredito': np.random.uniform(100000, 5000000, num_novos_clientes),
#     'score_atraso': np.random.uniform(0, 1, num_novos_clientes)
# })

# # Exibir as primeiras linhas da nova base de dados fictícia
# novos_clientes.to_excel('saidas/novos_clientes.xlsx')


In [164]:
# Verificar os tipos de dados
# print(solicitacoes_credito.dtypes)




